# 건축행정정보에 기반한 지하층 주택 실태조사 대상 추출

세움터의 건축데이터 민간개방 시스템에서 제공하는 대용량 원시데이터 기반으로
반지하 등 재해취약주택 실태조사를 위한
지하층 주택 실태조사 대상 목록을 추출

사용 데이터: 건축물대장(2022년 07월)

2022년 9월 현재 최신 데이터

In [2]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


## 건축물대장 데이터 구조 분석

In [3]:
ddf_floor: dd.DataFrame = dd.read_parquet("data/floor", calculate_divisions=True)
ddf_floor.head(1).transpose()


관리_건축물대장_PK,11110-1000
대지_위치,서울특별시 종로구 구기동 221번지
도로명_대지_위치,서울특별시 종로구 비봉5길 12
건물_명,None
시군구_코드,11110
법정동_코드,18200
대지_구분_코드,0
번,0221
지,0000
특수지_명,None
블록,None


In [ ]:
# len(ddf_floor), ddf_floor.index.nunique().compute()


총 7,889,941개 동 건축물의 20,563,496개 층 데이터

In [5]:
ddf_bldrgst: dd.DataFrame = dd.read_parquet("data/bldrgst", calculate_divisions=True)
ddf_bldrgst.head(1).transpose()


관리_건축물대장_PK,11110-1
관리_상위_건축물대장_PK,None
대장_구분_코드,1
대장_구분_코드_명,일반
대장_종류_코드,1
대장_종류_코드_명,총괄표제부
대지_위치,서울특별시 종로구 관철동 12-1번지
도로명_대지_위치,서울특별시 종로구 종로14길 20
건물_명,None
시군구_코드,11110
법정동_코드,13500


In [6]:
ddf_bldrgst.known_divisions


True

일반건축물 일반건축물대장 7,276,283개 동,
집합건축물 표제부 613,658개 동

In [7]:
ddf_title: dd.DataFrame = dd.read_parquet("data/title", calculate_divisions=True)
with pd.option_context("display.max_rows", 100, "display.max_columns", 100):
    display(ddf_title.head().transpose())


관리_건축물대장_PK,11110-1000,11110-10001,11110-10002,11110-10004,11110-10005
대장_구분_코드,2,1,1,1,1
대장_구분_코드_명,집합,일반,일반,일반,일반
대장_종류_코드,3,2,2,2,2
대장_종류_코드_명,표제부,일반건축물,일반건축물,일반건축물,일반건축물
대지_위치,서울특별시 종로구 구기동 221번지,서울특별시 종로구 평창동 454-3번지,서울특별시 종로구 종로4가 175-3번지,서울특별시 종로구 충신동 45-1번지,서울특별시 종로구 홍지동 68-1번지
도로명_대지_위치,서울특별시 종로구 비봉5길 12,서울특별시 종로구 평창31길 40,,서울특별시 종로구 율곡로19가길 10,서울특별시 종로구 홍지문길 27-12
건물_명,None,None,None,None,None
시군구_코드,11110,11110,11110,11110,11110
법정동_코드,18200,18300,16100,16700,18500
대지_구분_코드,0,0,0,0,0


In [8]:
ddf_title.columns


Index(['대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명', '대지_위치',
       '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명',
       '블록', '로트', '외필지_수', '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드',
       '새주소_본_번', '새주소_부_번', '동_명', '주_부속_구분_코드', '주_부속_구분_코드_명', '대지_면적(㎡)',
       '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)', '용적_률_산정_연면적(㎡)', '용적_률(%)', '구조_코드',
       '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명', '기타_용도', '지붕_코드', '지붕_코드_명',
       '기타_지붕', '세대_수(세대)', '가구_수(가구)', '높이(m)', '지상_층_수', '지하_층_수',
       '승용_승강기_수', '비상용_승강기_수', '부속_건축물_수', '부속_건축물_면적(㎡)', '총_동_연면적(㎡)',
       '옥내_기계식_대수(대)', '옥내_기계식_면적(㎡)', '옥외_기계식_대수(대)', '옥외_기계식_면적(㎡)',
       '옥내_자주식_대수(대)', '옥내_자주식_면적(㎡)', '옥외_자주식_대수(대)', '옥외_자주식_면적(㎡)', '허가_일',
       '착공_일', '사용승인_일', '허가번호_년', '허가번호_기관_코드', '허가번호_기관_코드_명', '허가번호_구분_코드',
       '허가번호_구분_코드_명', '호_수(호)', '에너지효율_등급', '에너지절감_율', '에너지_EPI점수',
       '친환경_건축물_등급', '친환경_건축물_인증점수', '지능형_건축물_등급', '지능형_건축물_인증점수', '생성_일자',
       '내진_설계_적용_여부', '내진_능력']

표제부(동별개요) 총 7,909,962개 동 중 7,889,941개 동에 층별개요 데이터 존재

In [105]:
ddf_exposarea: dd.DataFrame = dd.read_parquet(
    "data/exposarea", calculate_divisions=True
)
with pd.option_context("display.max_rows", 100, "display.max_columns", 100):
    display(ddf_exposarea.head().transpose())


관리_건축물대장_PK,11110-100178845,11110-100178845,11110-100178866,11110-100178866,11110-100178866
대장_구분_코드,2,2,2,2,2
대장_구분_코드_명,집합,집합,집합,집합,집합
대장_종류_코드,4,4,4,4,4
대장_종류_코드_명,전유부,전유부,전유부,전유부,전유부
대지_위치,서울특별시 종로구 무악동 82번지,서울특별시 종로구 무악동 82번지,서울특별시 종로구 신영동 105-25번지,서울특별시 종로구 신영동 105-25번지,서울특별시 종로구 신영동 105-25번지
도로명_대지_위치,서울특별시 종로구 통일로 246-10,서울특별시 종로구 통일로 246-10,서울특별시 종로구 세검정로6다길 3,서울특별시 종로구 세검정로6다길 3,서울특별시 종로구 세검정로6다길 3
건물_명,무악현대아파트,무악현대아파트,미엔빌,미엔빌,미엔빌
시군구_코드,11110,11110,11110,11110,11110
법정동_코드,18700,18700,18600,18600,18600
대지_구분_코드,0,0,0,0,0


## 지하층이 있는 다가구, 다중주택

일반건축물인 단독주택(다가구, 다중) 중 지하층 용도가 다가구, 다중이거나, 지하층이 있는 단독주택의 세대 수 또는 가구 수가 2 이상이거나 기타 용도에 다가구, 다중이 포함된 경우 실태조사 대상으로 선정.

### 용도 구분

In [10]:
ddf_title.주_용도_코드.dtype


dtype('O')

In [11]:
with pd.option_context("display.max_rows", 1000, "display.max_columns", 100):
    print(ddf_title.주_용도_코드.value_counts().compute())


01000    4354090
04000     674722
03000     584043
02000     552273
21000     432643
18000     404613
17000     392498
10000      88065
11000      47749
15000      45163
06000      39913
19000      36666
14000      36072
20000      35542
23000      23258
05000      15806
22000      13539
13000      13146
07000      12460
Z3000      11531
30000      10799
27000       9114
09000       7563
16000       5634
12000       4938
Z8000       4898
25000       4706
08000       3988
31000       3436
24000       3312
Z5000       2614
26000       1704
Z6000        534
29000        477
Z9000        399
JY           110
21101         87
06107         54
Z7001         44
02003         42
04010         27
18999         27
03001         22
28000         20
15200         20
창고            18
04203         14
04201         14
04100         13
12100         10
03009          9
07201          9
10004          7
07101          7
04999          7
14100          6
03101          5
03100          5
21100         

표제부는 절대 다수가 소분류 하위코드(단독, 다가구, 다중 등)가 아닌 대분류 상위코드(단독 등; 하위코드의 단독을 포함한 더 넓은 범위)로 기록되어 있음

In [12]:
with pd.option_context("display.max_rows", 1000, "display.max_columns", 100):
    print(ddf_floor.주_용도_코드.value_counts().compute())


01001    6141818
02001    2651919
01003    1926692
02003    1240170
03001     891815
18001     676363
04402     561274
04001     547662
21101     494078
17100     442874
03999     334841
17999     252584
02005     223796
04999     218296
04005     181135
14202     178955
21999     161773
02002     152448
18999     127346
03002     108094
03005     105219
04010     100834
04499      91783
15102      90148
14204      79716
18101      78936
01002      76422
17301      69243
10101      67608
14299      63796
11201      61828
20001      58023
06101      58008
03021      53049
04006      47967
03007      43172
03011      42719
03199      41577
10103      38641
10999      36363
10104      34595
10003      31144
23003      30063
19001      30012
15199      29584
10102      29286
04002      28101
04201      27548
17200      27309
04035      26102
02006      24765
10199      23534
09107      22202
03003      21800
03012      21377
02004      21205
16002      21140
20006      20935
11103      207

층별개요는 대다수에서 소분류 하위코드가 기재되어 있으므로 이를 사용

In [15]:
ddf_uground_floor = ddf_floor[(ddf_floor.층_구분_코드 == "10")]
ddf_uground_shared_floor = ddf_uground_floor[
    ddf_uground_floor.주_용도_코드.isin(
        [
            # "01000",  # 단독주택 대분류
            # "01001",  # 단독주택 소분류
            "01002",  # 다중주택 소분류
            "01003",  # 다가구주택 소분류
        ]
    )
]


표제부에 지하 층수가 1 이상이면서, 세대 수 또는 가구 수가 2 이상인 단독주택이거나, 기타 용도 텍스트에 "다가구", "다중"이 포함된 경우, 다가구, 다중 용도 지하층이 없더라도 지하층 주택에 포함

In [100]:
ddf_uground_title = ddf_title[ddf_title.지하_층_수 >= 1]
ddf_uground_title = ddf_uground_title[["세대_수(세대)", "가구_수(가구)", "주_용도_코드", "기타_용도"]]
ds_uground_shared_title = ddf_uground_title[
    (
        (
            ddf_uground_title.주_용도_코드.isin(
                [
                    "01000",  # 단독주택 대분류
                    "01001",  # 단독주택 소분류
                    "01002",  # 다중주택 소분류
                    "01003",  # 다가구주택 소분류
                ]
            )
        )
        & ((ddf_uground_title["세대_수(세대)"] >= 2) | (ddf_uground_title["가구_수(가구)"] >= 2))
    )
    | (ddf_uground_title.기타_용도.str.contains("|".join(["다가구", "다중"]), na=False))
]


### 추출

In [25]:
# df_uground_shared_floor = ddf_uground_shared_floor.head(1000, npartitions=10)


In [103]:
ddf_uground_shared = (
    ddf_uground_shared_floor.groupby(ddf_uground_shared_floor.index)
    .agg({"주_용도_코드": "first", "층_번호": "max", "층_구분_코드": "count"})
    .rename(
        columns={"주_용도_코드": "층_용도", "층_번호": "최저_지하_주택_층", "층_구분_코드": "지하_주택_층별개요_수"}
    )
)
ddf_uground_shared = ddf_uground_shared.join(
    ds_uground_shared_title.rename(
        columns={
            "세대_수(세대)": "표제부_세대_수",
            "가구_수(가구)": "표제부_가구_수",
            "주_용도_코드": "표제부_주_용도",
            "기타_용도": "표제부_기타_용도",
        }
    ),
    how="outer",
)
ddf_uground_shared = ddf_uground_shared.join(ddf_title)


In [104]:
with pd.option_context("display.max_rows", 1000, "display.max_columns", 100):
    display(ddf_uground_shared.head())


,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11110-100179949,NaN,<NA>,NaN,0,4,01000,"다가구주택,근린생활시설",1,일반,2,일반건축물,서울특별시 종로구 평창동 182-1번지,서울특별시 종로구 평창12길 13,None,11110,18300,0,0182,0001,None,None,None,0,111104100488,18301,0,13,<NA>,None,0,주건축물,169.00,100.04,59.20,490.91,332.77,196.91,21,철근콘크리트구조,철근콘크리트구조,01000,단독주택,"다가구주택,근린생활시설",10,(철근)콘크리트,(철근)콘크리트,0,4,13.500,4,1,0,0,0,0.0,490.91,0,0.0,0,0.0,0,0.0,5,0.0,20080109,20080211,20080701,2008,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20190806,1,None
11110-100192807,01003,1,3.0,0,10,01000,다가구주택,1,일반,2,일반건축물,서울특별시 종로구 구기동 55-17번지,서울특별시 종로구 홍지문길 88,None,11110,18200,0,0055,0017,None,None,None,0,111104100556,18201,0,88,0,None,0,주건축물,297.00,88.29,29.73,396.09,264.87,89.18,21,철근콘크리트구조,철근콘크리트구조,01000,단독주택,다가구주택,10,(철근)콘크리트,(철근)콘크리트,0,10,11.390,3,1,0,0,0,0.0,396.09,0,0.0,0,0.0,0,0.0,7,80.5,20120312,20120327,20120926,2012,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20190530,1,None
11110-100209165,NaN,<NA>,NaN,0,8,01000,단독주택(다가구주택),1,일반,2,일반건축물,서울특별시 종로구 숭인동 181-51번지,서울특별시 종로구 종로65길 42,갤러리,11110,17500,0,0181,0051,None,None,None,3,111104100370,17501,0,42,0,None,0,주건축물,232.16,127.47,54.91,599.58,456.73,196.73,21,철근콘크리트구조,철근콘크리트구조,01000,단독주택,단독주택(다가구주택),10,(철근)콘크리트,(철근)콘크리트,0,8,14.952,5,1,1,0,0,0.0,599.58,0,0.0,0,0.0,6,0.0,1,0.0,20160908,20170407,20180214,2016,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20191119,1,None
11110-10710,01003,1,1.0,0,5,01000,다가구용주택(5가구),1,일반,2,일반건축물,서울특별시 종로구 숭인동 254번지,서울특별시 종로구 종로58길 8-3,None,11110,17500,0,0254,0000,None,None,None,0,111104100359,17501,0,8,3,None,0,주건축물,181.80,0.00,0.00,296.19,0.00,0.00,11,벽돌구조,연와조,01000,단독주택,다가구용주택(5가구),10,(철근)콘크리트,평스라브,0,5,0.000,2,1,0,0,0,0.0,296.19,0,0.0,0,0.0,0,0.0,0,0.0,19920813,None,19921128,None,None,None,None,None,0,None,0.0,0,None,0,None,0,20200427,0,None
11110-11031,01003,1,1.0,0,10,01000,다가구용단독주택(10),1,일반,2,일반건축물,서울특별시 종로구 창신동 23-520번지,서울특별시 종로구 창신11길 12,None,11110,17400,0,0023,0520,None,None,None,0,111104100431,17401,0,12,0,None,0,주건축물,179.50,102.17,56.91,406.18,0.00,155.04,11,벽돌구조,연와조,01000,단독주택,다가구용단독주택(10),10,(철근)콘크리트,평스라브,0,10,8.750,3,1,0,0,0,0.0,406.18,0,0.0,0,0.0,0,0.0,3,34.5,19940728,19940808,19941215,None,None,None,None,None,0,None,0.0,0,None,0,None,0,20200427,0,None


다음은 층별개요를 전유부가 아닌 호실 정보로 오용한 사례.

In [104]:
with pd.option_context("display.max_rows", 1000, "display.max_columns", 100):
    display(ddf_floor[ddf_floor.층_구분_코드 == "10"].loc["41115-181747"].compute())


,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(계단실 복도 등),42.81,0,주건축물,0,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(204호),24.75,0,주건축물,0,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,2,지2,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,"단독주택(주차장,계단실)",92.31,0,주건축물,None,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(202호),23.10,0,주건축물,0,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(201호),23.10,0,주건축물,0,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(206호),24.75,0,주건축물,0,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(205호),24.75,0,주건축물,0,20200312
41115-181747,경기도 수원시 팔달구 교동 6-104번지,경기도 수원시 팔달구 향교로137번길 39,None,41115,13200,0,0006,0104,None,None,None,411154328418,13202,0,39,<NA>,None,10,지하,1,지1,21,철근콘크리트구조,철근콘크리트구조,01003,다가구주택,다가구주택(203호),23.10,0,주건축물,0,20200312


## 지하 전유부가 있는 다세대, 연립주택

집합건축물인 공동주택(다세대, 연립) 중 지하층 전유부의 용도가 주택이거나, 건물동의 층별 용도가 다세대, 연립이거나, 지하층이 있는 건물동의 기타 용도에 "다세대", "연립"이 포함된 경우 실태조사 대상으로 선정.

### 전유부의 전유공용면적 부분별 용도

In [242]:
ddf_exposarea


,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,호_명,층_구분_코드,층_구분_코드_명,층_번호,전유_공용_구분_코드,전유_공용_구분_코드_명,주_부속_구분_코드,주_부속_구분_코드_명,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),생성_일자
npartitions=467,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11110-100178845,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,Int64,Int64,object,object,object,object,Int64,object,object,object,object,object,object,object,object,object,object,object,float64,object
11110-62856,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50130-100236698,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50130-88840,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [243]:
ddf_uground_exposarea = ddf_exposarea[(ddf_exposarea.층_구분_코드 == "10")]


In [244]:
ddf_uground_exposarea.주_용도_코드.head(1000).value_counts().head()


04001    271
02003    128
04999     90
03001     78
04402     65
Name: 주_용도_코드, dtype: int64

전유부 주 용도가 잘못 기재된 경우를 포괄하기 위하여, 단독, 다가구, 다중을 포함하여 용도 필터링.

In [245]:
ddf_uground_multi_exposarea = ddf_uground_exposarea[
    ddf_uground_exposarea.주_용도_코드.isin(
        [
            "01001",  # 단독주택 소분류
            "01002",  # 다중주택 소분류
            "01003",  # 다가구주택 소분류
            "02002",  # 연립주택 소분류
            "02003",  # 다세대주택 소분류
        ]
    )
]


In [246]:
ddf_uground_multi_exposarea.head()


,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,...,주_부속_구분_코드_명,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),생성_일자
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,
11110-100179510,2,집합,4,전유부,서울특별시 종로구 평창동 523-96번지,서울특별시 종로구 평창21길 76,None,11110,18300,0,...,주건축물,지1,21,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,182.38,20210114
11110-100179510,2,집합,4,전유부,서울특별시 종로구 평창동 523-96번지,서울특별시 종로구 평창21길 76,None,11110,18300,0,...,주건축물,지2,21,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,주차장,47.04,20210114
11110-100179510,2,집합,4,전유부,서울특별시 종로구 평창동 523-96번지,서울특별시 종로구 평창21길 76,None,11110,18300,0,...,주건축물,각층,21,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,계단실외,26.24,20210114
11110-100184444,2,집합,4,전유부,서울특별시 종로구 숭인동 181-176번지,서울특별시 종로구 숭인동1길 47,신성아트빌,11110,17500,0,...,주건축물,각층,21,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,계단실,8.82,20181001
11110-100184444,2,집합,4,전유부,서울특별시 종로구 숭인동 181-176번지,서울특별시 종로구 숭인동1길 47,신성아트빌,11110,17500,0,...,주건축물,지1,21,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택(1세대),61.50,20181001


In [247]:
ddf_uground_multi_exposarea.주_용도_코드.head(10000, npartitions=10).value_counts()


02003    5304
02002    4689
01001       7
Name: 주_용도_코드, dtype: int64

In [248]:
ddf_bldrgst.loc["11110-100179510"].compute()


,관리_상위_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,...,새주소_지상지하_코드,새주소_본_번,새주소_부_번,지역_코드,지구_코드,구역_코드,지역_코드_명,지구_코드_명,구역_코드_명,생성_일자
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,
11110-100179510,11110-100179509,2,집합,4,전유부,서울특별시 종로구 평창동 523-96번지,서울특별시 종로구 평창21길 76,None,11110,18300,...,0,76,0,None,None,None,None,None,None,20210114


In [249]:
ddf_title.loc["11110-100179509"].compute()


,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,...,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,
11110-100179509,2,집합,3,표제부,서울특별시 종로구 평창동 523-96번지,서울특별시 종로구 평창21길 76,None,11110,18300,0,...,None,0.0,0,None,0,None,0,20200427,0,None


전유공유부분 데이터의 PK는 전유부의 PK이며, 마스터 데이터의 상위 PK가 건물동의 PK임.

### 층별 용도

In [226]:
ddf_uground_floor = ddf_floor[(ddf_floor.층_구분_코드 == "10")]
ddf_uground_multi_floor = ddf_uground_floor[
    ddf_uground_floor.주_용도_코드.isin(
        [
            "02002",  # 연립주택 소분류
            "02003",  # 다세대주택 소분류
        ]
    )
]


### 표제부 기타 용도

In [258]:
ddf_uground_title = ddf_title[ddf_title.지하_층_수 >= 1]
ddf_uground_title = ddf_uground_title[["세대_수(세대)", "가구_수(가구)", "주_용도_코드", "기타_용도"]]
ddf_uground_multi_title = ddf_uground_title[
    ddf_uground_title.기타_용도.str.contains("|".join(["다세대", "연립"]), na=False)
]


### 추출

In [294]:
# 전유부 단위 집계

ddf_uground_multi_expos = (
    ddf_uground_multi_exposarea[["주_용도_코드", "층_번호", "층_구분_코드"]]
    .groupby(ddf_uground_multi_exposarea.index)
    .agg({"주_용도_코드": "first", "층_번호": "max", "층_구분_코드": "count"})
    .rename(
        columns={
            "주_용도_코드": "전유부_용도",
            "층_번호": "최저_지하_주택_전유부_층",
            "층_구분_코드": "지하_주택_전유공용면적_수",
        }
    )
    .join(ddf_bldrgst.관리_상위_건축물대장_PK)
)


In [295]:
# 건물동 단위 집계
ddf_uground_multi = (
    ddf_uground_multi_expos.groupby("관리_상위_건축물대장_PK")
    .agg(
        {
            "전유부_용도": "first",
            "최저_지하_주택_전유부_층": "max",
            "지하_주택_전유공용면적_수": "count",
        }
    )
    .rename(
        columns={
            "지하_주택_전유공용면적_수": "지하_주택_전유부_수",
        }
    )
)
ddf_uground_multi.index = ddf_uground_multi.index.rename("관리_건축물대장_PK")

ddf_uground_multi = ddf_uground_multi.join(
    [
        (
            ddf_uground_multi_floor[["주_용도_코드", "층_번호", "층_구분_코드"]]
            .groupby(ddf_uground_multi_floor.index)
            .agg({"주_용도_코드": "first", "층_번호": "max", "층_구분_코드": "count"})
            .rename(
                columns={
                    "주_용도_코드": "층_용도",
                    "층_번호": "최저_지하_주택_층",
                    "층_구분_코드": "지하_주택_층별개요_수",
                }
            )
        ),
        (
            ddf_uground_multi_title.rename(
                columns={
                    "세대_수(세대)": "표제부_세대_수",
                    "가구_수(가구)": "표제부_가구_수",
                    "주_용도_코드": "표제부_주_용도",
                    "기타_용도": "표제부_기타_용도",
                }
            )
        ),
    ],
    how="outer",
)

ddf_uground_multi = ddf_uground_multi.join(ddf_title)


In [299]:
with pd.option_context("display.max_rows", 10, "display.max_columns", 100):
    display(ddf_uground_multi.head())


,전유부_용도,최저_지하_주택_전유부_층,지하_주택_전유부_수,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11110-100188058,NaN,<NA>,NaN,02003,2,3.0,<NA>,<NA>,NaN,NaN,2,집합,3,표제부,서울특별시 종로구 평창동 361-6번지,서울특별시 종로구 평창13길 10,오보에힐스,11110,18300,0,0361,0006,None,None,None,0,111104100489,18301,0,10,<NA>,지하주차장1,1,부속건축물,0.0,0.00,0.00,369.8000,0.00,0.00,21,철근콘크리트구조,철근콘크리트구조,02000,공동주택,"지하주차장,기계실,창고,경비실,기사대기실,쓰레기분리수거실",10,(철근)콘크리트,(철근)콘크리트평지붕,0,0,0.00,0,2,0,0,0,0.0,369.8000,0,0.0,0,0.0,0,0.00,0,0.0,20080502,20080528,20100625,2008,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20200427,0,None
11110-100196815,02003,1,1.0,02003,1,2.0,10,0,02000,다세대주택(10세대),2,집합,3,표제부,서울특별시 종로구 부암동 129-19번지,서울특별시 종로구 백석동길 286-7,부암로자벨,11110,18400,0,0129,0019,None,None,None,1,111104100094,18401,0,286,7,부암로자벨B동,0,주건축물,1507.0,226.57,15.03,863.7900,588.69,39.06,21,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택(10세대),10,(철근)콘크리트,금속기와마감,10,0,13.40,3,1,0,0,0,0.0,863.7900,0,0.0,0,0.0,16,411.07,0,0.0,20121022,20130315,20140103,2012,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20190530,1,None
11110-100205257,NaN,<NA>,NaN,02003,1,2.0,<NA>,<NA>,NaN,NaN,2,집합,3,표제부,서울특별시 종로구 청운동 57-27번지,서울특별시 종로구 자하문로33길 31,None,11110,10100,0,0057,0027,None,None,None,6,111104100285,10101,0,31,<NA>,주건축물제1동,0,주건축물,231.4,114.14,49.33,459.1000,347.04,149.97,21,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택,10,(철근)콘크리트,경사지붕,8,0,16.65,4,1,1,0,0,0.0,459.1000,0,0.0,0,0.0,4,46.17,2,23.5,20160211,20160512,20161107,2016,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20190530,1,None
11110-100214305,NaN,<NA>,NaN,NaN,<NA>,NaN,11,0,02000,다세대주택,2,집합,3,표제부,서울특별시 종로구 혜화동 5-45번지,서울특별시 종로구 혜화로16길 5,젬스톤빌리지,11110,16900,0,0005,0045,None,None,None,0,111104100542,16901,0,5,<NA>,젬스톤빌리지,0,주건축물,376.9,218.04,57.85,835.0300,563.20,149.43,21,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10,(철근)콘크리트,(철근)콘크리트,11,0,11.82,4,1,1,0,0,0.0,835.0300,0,0.0,0,0.0,8,244.63,1,11.5,20180424,20180530,20181212,2018,3000082,건축과,1101,신축허가,1,None,0.0,0,None,0,None,0,20200120,1,내진설계범주(D) 동적해석법
11110-100225402,NaN,<NA>,NaN,02002,1,7.0,<NA>,<NA>,NaN,NaN,2,집합,3,표제부,서울특별시 종로구 구기동 267번지,서울특별시 종로구 비봉길 28,쌍용 더 플래티넘 종로 구기동,11110,18200,0,0267,0000,None,None,None,0,111104100122,18201,0,28,0,105동,1,부속건축물,5912.3,0.00,0.00,2967.8539,0.00,0.00,21,철근콘크리트구조,철근콘크리트구조,02000,공동주택,"지하주차장,경비실,관리사무소,MDF실,방재실,재활용보관창고,전기실,발전기실,통신실",10,(철근)콘크리트,(철근)콘크리트,0,0,4.20,0,1,1,0,0,0.0,2967.8539,0,0.0,0,0.0,0,0.00,0,0.0,20191121,20200312,20211015,2019,3000148,도시개발과,2101,주택건설사업계획승인,0,None,0.0,0,None,0,None,0,20211022,0,None


## 저장

In [ ]:
with ProgressBar():
    ddf_uground_shared.to_parquet("output/uground_shared")
    ddf_uground_shared.to_csv("output/uground_shared.csv", single_file=True)


In [300]:
with ProgressBar():
    ddf_uground_multi.to_parquet("output/uground_multi")
    ddf_uground_multi.to_csv("output/uground_multi.csv", single_file=True)


[########################################] | 100% Completed | 451.94 s
[########################################] | 100% Completed | 428.94 s


## 결과 검토

In [301]:
ddf_uground_shared = dd.read_parquet("output/uground_shared")
ddf_uground_multi = dd.read_parquet("output/uground_multi")


In [302]:
len(ddf_uground_shared)


240075

In [303]:
ddf_uground_shared.groupby(ddf_uground_shared.index).count().loc[
    lambda x: x.대장_구분_코드 >= 2
].compute()


,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,...,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,


In [304]:
ddf_uground_shared.groupby(ddf_uground_shared.index).count().head(10)


,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,...,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,
11110-100179949,0,0,0,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-100192807,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-100209165,0,0,0,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-10710,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-11031,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-11346,0,0,0,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-11415,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-11535,1,1,1,0,0,0,0,1,1,1,...,0,1,1,0,1,0,1,1,1,0
11110-11690,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,1,0,1,1,1,0


In [305]:
ddf_uground_shared.지하_주택_층별개요_수.sum().compute()


181163.0

다가구주택, 다중주택: 240,075동 (주택 용도 지하 층별개요 181,163건)

In [306]:
len(ddf_uground_multi)


124701

In [307]:
ddf_uground_multi.groupby(ddf_uground_multi.index).count().loc[
    lambda x: x.대장_구분_코드 >= 2
].compute()


,전유부_용도,최저_지하_주택_전유부_층,지하_주택_전유부_수,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,...,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,


In [308]:
ddf_uground_multi.지하_주택_전유부_수.sum().compute()


113803.0

다세대주택, 연립주택: 124,701동 (주택 용도 지하 전유부 113,803건)

총 364,776동이 실태조사 대상